Using pyfasst NMF (doesn't work)

In [16]:
import pyfasst.audioModel as am

filename='../wav/timit_denoising_mixture.wav'
# initialize the model
model = am.MultiChanNMFInst_FASST(
	audio=filename,
	nbComps=2, nbNMFComps=32, spatial_rank=1, iter_num=50)
model.makeItConvolutive()

model.initializeConvParams(initMethod='demix')
model.estim_param_a_post_model()
model.separate_spat_comps(dir_results='nmf-results/')


IndexError: too many indices for array

Sk-learn NMF (doesn't work)

In [15]:
from sklearn.decomposition import NMF

filename='../wav/timit_denoising_mixture.wav'
x, fs = librosa.load(filename)

x[0] = 1 # hack to prevent automatic gain scaling
in_audio = IPython.display.Audio(x, rate=fs)
IPython.display.display(in_audio)

S = librosa.stft(x)
X = numpy.absolute(S)

model = NMF(n_components=2, init='random', random_state=0)
model.fit(X)
components = model.components_
a1 = IPython.display.Audio(components[0], rate=fs)
IPython.display.display(a1)
a2 = IPython.display.Audio(components[1], rate=fs)
IPython.display.display(a2)

Using NMF from librosa (doesn't work)

In [12]:
import numpy, scipy, matplotlib.pyplot as plt, sklearn, IPython.display, librosa, urllib

filename='../wav/timit_denoising_mixture.wav'
x, fs = librosa.load(filename)

x[0] = 1 # hack to prevent automatic gain scaling
in_audio = IPython.display.Audio(x, rate=fs)
IPython.display.display(in_audio)

n_components = 2
W, H = librosa.decompose.decompose(X, n_components=n_components, sort=True)

reconstructed_signal = scipy.zeros(len(x))
for n in range(n_components):
    Y = scipy.outer(W[:,n], H[n])*numpy.exp(1j*numpy.angle(S))
    y = librosa.istft(Y)
    reconstructed_signal[:len(y)] += y
    IPython.display.display( IPython.display.Audio(y, rate=fs) )
a = IPython.display.Audio(reconstructed_signal, rate=fs)
IPython.display.display(a)



Evaluation of SDR, SIR, SAR

In [36]:
import mir_eval, IPython.display
import numpy as np


# Noise Separation
sig1_wav = './wav/timit_denoising_groundtruth_speech.wav'
sig2_wav = './wav/timit_denoising_groundtruth_noise.wav'
sep_sig1_wav = './wav/timit_denoising_mixture_DENOISED.wav'
sep_sig2_wav = './wav/timit_denoising_separated_noise.wav'
mix_wav = './wav/timit_denoising_mixture.wav'
(groundtruth_noise, fs_groundtruth_noise) = mir_eval.io.load_wav('../../wav/timit_denoising_groundtruth_noise.wav')
(groundtruth_signal, fs_groundtruth_signal) = mir_eval.io.load_wav('../../wav/timit_denoising_groundtruth_speech.wav')
(separated_signal_nmf, fs_separated_signal) = mir_eval.io.load_wav('../../wav/timit_denoising_mixture_DENOISED.wav')
(separated_signal, fs_separated_noise) = mir_eval.io.load_wav('../../wav/timit_denoising_separated_speech.wav')
(separated_noise, fs_separated_noise) = mir_eval.io.load_wav('../../wav/timit_denoising_separated_noise.wav')
(mixture, fs_mixture) = mir_eval.io.load_wav('../../wav/timit_denoising_mixture.wav')

# mixed speech
(speech_mixture, speech_fs_mixture) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_mixture.wav');



# Speech Separation
print "Metrics for Voice Separation"
IPython.display.display( IPython.display.Audio(speech_mixture, rate=fs))

(malevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_groundtruth_male.wav')
(femalevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_groundtruth_female.wav')
(separated_malevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_mixture_DENOISED_for_male.wav')
(separated_femalevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_mixture_DENOISED_for_female.wav')
size = min( len(malevoice), len(femalevoice), len(separated_malevoice), len(separated_femalevoice) )
reference_sources = np.array([malevoice[0:size], femalevoice[0:size]])
estimated_sources = np.array([separated_malevoice[0:size], separated_femalevoice[0:size]])
evals = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
print "NMF"
IPython.display.display( IPython.display.Audio(separated_malevoice, rate=fs))
IPython.display.display( IPython.display.Audio(separated_femalevoice, rate=fs))
print "SDR male: ", evals[0][0]
print "SDR female: ", evals[0][1]

print "SIR male: ", evals[1][0]
print "SIR female: ", evals[1][1]

print "SAR male: ", evals[2][0]
print "SAR female:", evals[2][1]
print




(malevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_groundtruth_male.wav')
(femalevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_groundtruth_female.wav')
(separated_malevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_separated_male.wav')
(separated_femalevoice, fs) = mir_eval.io.load_wav('../../wav/speech-separation/timit_speech_separation_separated_female.wav')
size = min( len(malevoice), len(femalevoice), len(separated_malevoice), len(separated_femalevoice) )
reference_sources = np.array([malevoice[0:size], femalevoice[0:size]])
estimated_sources = np.array([separated_malevoice[0:size], separated_femalevoice[0:size]])
evals = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
print "DRNN"
IPython.display.display( IPython.display.Audio(separated_malevoice, rate=fs))
IPython.display.display( IPython.display.Audio(separated_femalevoice, rate=fs))
print "SDR male: ", evals[0][0]
print "SDR female: ", evals[0][1]

print "SIR male: ", evals[1][0]
print "SIR female: ", evals[1][1]

print "SAR male: ", evals[2][0]
print "SAR female:", evals[2][1]
print


(mixture, fs_mixture) = mir_eval.io.load_wav('../../mixed/noise1_voice1.wav')
print "Metrics for Noise Separation"
IPython.display.display( IPython.display.Audio(mixture, rate=fs))

# SOX noise reduction
(groundtruth_voice, fs_groundtruth_voice) = mir_eval.io.load_wav('../../voice/voice1.wav')
(separated_voice, fs_separated_voice) = mir_eval.io.load_wav('../../clean/noise1_voice1_cleaned.wav')
(noise, fs_noise) = mir_eval.io.load_wav('../../noise/noise1.wav')
size = min(len(groundtruth_voice), len(separated_voice), len(mixture), len(noise))


print "Metrics for SOX noise reduction"
IPython.display.display( IPython.display.Audio(separated_voice, rate=fs))
reference_sources = np.array([groundtruth_voice[0:size], noise[0:size]]) #, groundtruth_noise[0:size]])
estimated_sources = np.array([separated_voice[0:size], noise[0:size]]) #, separated_noise[0:size]])

(sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
print "sdr: ", sdr[0]
print "sir: ", sir[0]
print "sar: ", sar[0]
print



print "Metrics for NMF Separation"
IPython.display.display( IPython.display.Audio(separated_signal_nmf, rate=fs))
size = min(len(groundtruth_noise), len(groundtruth_signal), len(separated_signal), len(separated_noise), len(separated_signal_nmf))
reference_sources = np.array([groundtruth_signal[0:size], groundtruth_noise[0:size]])
estimated_sources = np.array([separated_signal_nmf[0:size], separated_noise[0:size]])

(sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
print "sdr: ", sdr[0]
print "sir: ", sir[0]
print "sar: ", sar[0]
print

print "Metrics for Joint Optimization of Masks and DRNN Separation"
IPython.display.display( IPython.display.Audio(separated_signal, rate=fs))
size = min(len(groundtruth_noise), len(groundtruth_signal), len(separated_signal), len(separated_noise), len(separated_signal_nmf))
reference_sources = np.array([groundtruth_signal[0:size], groundtruth_noise[0:size]])
estimated_sources = np.array([separated_signal[0:size], separated_noise[0:size]])

(sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources)
print "sdr: ", sdr[0]
print "sir: ", sir[0]
print "sar: ", sar[0]




Metrics for Voice Separation


NMF


SDR male:  3.50865352903
SDR female:  3.97700096515
SIR male:  7.75295392215
SIR female:  6.14096299584
SAR male:  6.23267095843
SAR female: 8.98478760009

DRNN


SDR male:  6.97909121975
SDR female:  7.94929320711
SIR male:  9.83376818446
SIR female:  13.9362631276
SAR male:  10.580103125
SAR female: 9.38194914508

Metrics for Noise Separation


Metrics for SOX noise reduction


sdr:  2.44592309098
sir:  8.26068717292
sar:  4.37031607126

Metrics for NMF Separation


sdr:  4.97792577214
sir:  6.60361048799
sar:  10.8915134067

Metrics for Joint Optimization of Masks and DRNN Separation


sdr:  11.6562526279
sir:  17.1807523945
sar:  13.1667882823
